# Binary image classifier with:
* TPU / Multi-GPU ready code
* Dataset created from directories with separate classes
* Preprocessing and augmentation as a Keras layer in dataset preprocessor
* Transfer learning based on ResNET

* Builds on:

https://www.kaggle.com/code/donkeys/keras-binary-cats-dogs-resnet-98

https://towardsdatascience.com/a-comprehensive-guide-to-training-cnns-on-tpu-1beac4b0eb1c

In [ ]:
# Initial imports
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from tensorflow.keras import layers
import tensorflow_hub as hub
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import PIL
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.models import Sequential, Model, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                         BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)

In [ ]:
try: 
    # For use with TPU:

    # Detect TPUs
    
    # Locate TPUs on the network
    # tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    
    # TPUStrategy contains the necessary distributed training code that will work on TPUs 
    # with their 8 compute cores
    # strategy = tf.distribute.TPUStrategy(tpu)
    
    # Multi GPU training
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"]) #, "/gpu:1"])

except ValueError: # If TPU or GPU is not available
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

In [ ]:
print(f'Number of accelerators: {strategy.num_replicas_in_sync}')

In [ ]:
!pwd

In [ ]:
PATH_IMAGES = './data/PetImages'

In [ ]:
!ls $PATH_IMAGES

In [ ]:
BATCH_SIZE = 32 * strategy.num_replicas_in_sync

AUTOTUNE = tf.data.AUTOTUNE

# This is related to the feature size optimization, a multiple of 128 required for TPU
IMG_SIZE = 128 * 2

In [ ]:
train_val_dir = PATH_IMAGES
train_val_cat_files = os.listdir(PATH_IMAGES + '/Cat')
train_val_dog_files = os.listdir(PATH_IMAGES + '/Dog')

# Add a set for final model testing if needed
# test_dir = 

In [ ]:
TRAIN_TOTAL = len(train_val_cat_files) + len(train_val_dog_files)

In [ ]:
CAT = 'cat'
DOG = 'dog'

In [ ]:
labels = []
df_data = pd.DataFrame()

In [ ]:
TRAIN_TOTAL

In [ ]:
%%time
idx = 0
img_sizes = []
file_dir = []
files_str = []
widths = np.zeros(TRAIN_TOTAL, dtype=int)
heights = np.zeros(TRAIN_TOTAL, dtype=int)
aspect_ratio = np.zeros(TRAIN_TOTAL)

for filename in train_val_cat_files:
    labels.append(CAT)
    filename_str = f'{PATH_IMAGES}/Cat/{filename}'
    files_str.append(filename_str)
    img = PIL.Image.open(filename_str).convert('RGB')
    file_dir.append(f'{PATH_IMAGES}/Cat/')
    img_size = img.size
    img_sizes.append(img_size)
    widths[idx] = img_size[0]
    heights[idx] = img_size[1]
    aspect_ratio[idx] = img_size[0]/img_size[1]
    
    # We can resize already here if we want
    #img = img.resize((IMG_SIZE, IMG_SIZE))
    #img.save(filename_str)
    
    idx+=1
    
for filename in train_val_dog_files:
    labels.append(DOG)
    filename_str = f'{PATH_IMAGES}/Dog/{filename}'
    files_str.append(filename_str)
    img = PIL.Image.open(filename_str).convert('RGB')
    file_dir.append(f'{PATH_IMAGES}/Dog/')
    img_size = img.size
    img_sizes.append(img_size)
    widths[idx] = img_size[0]
    heights[idx] = img_size[1]
    aspect_ratio[idx] = img_size[0]/img_size[1]
    
    # We can resize already here if we want
    #img = img.resize((IMG_SIZE, IMG_SIZE))
    #img.save(filename_str)
    
    idx+=1

In [ ]:
file_list = train_val_cat_files + train_val_dog_files

In [ ]:
len(labels)

# Creating dataset dataframe from directory

In [ ]:
df_data['filename'] = file_list
df_data['filedir'] = file_dir
df_data['cat_or_dog'] = labels
df_data['files_str'] = files_str
label_encoder = LabelEncoder()
df_data['cd_label'] = label_encoder.fit_transform(df_data['cat_or_dog'])
df_data["size"] = img_sizes
df_data["width"] = widths
df_data["height"] = heights
df_data["aspect_ratio"] = aspect_ratio
# df_data.head()

In [ ]:
# Sorting by aspect ratio to detect some edge case shapes
df_sorted = df_data.sort_values(by='aspect_ratio')

In [ ]:
def plot_first_9(df_to_plot):
    plt.figure(figsize=[30, 30])
    for x in range(9):
        filename = df_to_plot.iloc[x]['filename']
        path_to_plot = df_to_plot.iloc[x]['filedir'] + df_to_plot.iloc[x]['filename']
        img = PIL.Image.open(path_to_plot)
        print(filename)
        plt.subplot(3, 3, x + 1)
        plt.imshow(img)
        title_str = filename+" "+str(df_to_plot.iloc[x].aspect_ratio)
        plt.title(title_str)

In [ ]:
# Dropping wrong samples
df_sorted=df_sorted[:-3]

In [ ]:
def convert_to_image(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    # ResNET may not need to work with floats
    #img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

In [ ]:
def convert_to_images_labels(filename, label):
    return convert_to_image(filename), label

# Keras preprocessing layer for the ResNET case and Imagenet dataset

In [ ]:
#from keras.applications.resnet50 import preprocess_input as resnet_preprocess
from keras.applications.imagenet_utils import preprocess_input as resnet_preprocess

def resnet_preprocessor(img, label):
    return resnet_preprocess(img), label

In [ ]:
def prepare_for_training(ds, data_augement_fn=None):
    ds = ds.map(convert_to_images_labels,
               num_parallel_calls=AUTOTUNE)
    
    # ResNET-50 preprocessing
    
    ds = ds.map(resnet_preprocessor,
               num_parallel_calls=AUTOTUNE)

    # Only for datasets fiting in memmory
    # ds = ds.cache() # Important to do before data aug
    
    # Big buffer size preferred
    ds = ds.shuffle(buffer_size=2048)
    
    # Infinite dataset
    ds = ds.repeat()
    
    ds = ds.batch(BATCH_SIZE)
    
    # Apply data augmentation
    if data_augement_fn:
        ds = ds.map(data_augement_fn,
                   num_parallel_calls=AUTOTUNE)
        
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds

In [ ]:
def plot_learning_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(acc))
    
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Trainig and validation accuracy')
    plt.legend(loc=0)
    plt.figure();
    
    plt.show();

In [ ]:
# df_sorted

In [ ]:
# Randomizing dataframe rows
df_sorted=df_sorted.sample(frac=1)
df_sorted=df_sorted.sample(frac=1)

In [ ]:
train_val_split=int(0.25*len(df_sorted))

In [ ]:
df_train=df_sorted[:-train_val_split]

In [ ]:
df_val=df_sorted[-train_val_split:]

In [ ]:
train_files_ds = tf.data.Dataset.from_tensor_slices(df_train['files_str'])
val_files_ds = tf.data.Dataset.from_tensor_slices(df_val['files_str'])

train_labels_ds = tf.data.Dataset.from_tensor_slices(df_train['cd_label'])
val_labels_ds = tf.data.Dataset.from_tensor_slices(df_val['cd_label'])


In [ ]:
train_list_ds = tf.data.Dataset.zip((train_files_ds, train_labels_ds))

val_list_ds = tf.data.Dataset.zip((val_files_ds, val_labels_ds))

In [ ]:
# See a piece of the files dataset with labels
for sample in train_list_ds.take(5):
    print(sample[0].numpy(), sample[1].numpy())

# Converting files TF dataset into pictures dataset|

In [ ]:
train_ds = prepare_for_training(train_list_ds)
val_ds = prepare_for_training(val_list_ds)

In [ ]:
train_ds.options

# Test preprocessing effect directly from the created dataset batch

In [ ]:
def plot_batch_9(ds):
    aux_ds=iter(ds)
    plt.clf()
    plt.figure(figsize=[30, 30])
    batch = next(aux_ds)
    for n in range(9):
        plt.subplot(3, 3, n+1)
        plt.imshow(batch[0][n])
        
    plt.show()  

In [ ]:
plot_batch_9(train_ds)

In [ ]:
batch = next(iter(train_ds))

# For an infinite dataset training (ds.repeat()) one has to set 
* steps_per_epoch
* validation_steps

# Note: remember to tune batch size for TPU and learning rate accordingly to the (large) batch size (not done here)

In [ ]:
steps_per_epoch = math.ceil(len(train_list_ds)/BATCH_SIZE)
validation_steps = math.ceil(len(val_list_ds)/BATCH_SIZE)

# ResNET-50 Transfer Learning

In [ ]:
from keras.applications.resnet50 import ResNet50

* Model creation function allows to specify how many layers are to be kept frozen

In [ ]:
def define_model(trainable_layers_count, show_summary=False):
    
    input_tensor = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model = ResNet50(include_top=False,
                         #weights=None,
                          weights='imagenet',
                         input_tensor=input_tensor)
    # base_model.load_weights('./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
    if trainable_layers_count=='all':
        for layer in base_model.layers:
            layer.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False
            
        for layer in base_model.layers[-trainable_layers_count:]:
            layer.trainable = True
        
    print('Base model has {} layers'.format(len(base_model.layers)))
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(5e-4))(x)
    x = Dropout(0.5)(x)
    final_outpu = Dense(1, activation='sigmoid', name='final_output')(x)
    
    model = Model(input_tensor, final_outpu)
    
    if show_summary:
        model.summary()
        
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'],
                 steps_per_execution=32)
    
    return model

# Creating useful callback functions

In [ ]:
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler, 
                            EarlyStopping, ReduceLROnPlateau, CSVLogger)

checkpoint = ModelCheckpoint('./working/Resnet50_best.h5', monitor='val_loss',
                            verbose=1, save_best_only=True, mode='min', save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                  verbose=1, mode='auto', epsilon=0.0001)

early = EarlyStopping(monitor='val_loss',
                      mode='min',
                     patience=7)

csv_logger = CSVLogger(filename='./working/training_log_csv',
                      separator=',',
                      append=True)

callbacks_list = [checkpoint, csv_logger, early]

# Creating model in the distributed strategy scope

In [ ]:
with strategy.scope():
    model = define_model(3, show_summary=True)

# Training

In [ ]:
history = model.fit(train_ds,
                   steps_per_epoch=steps_per_epoch,
                   epochs=5,
                   validation_data=val_ds,
                   validation_steps=validation_steps,
                   verbose=1,
                   callbacks=callbacks_list)

# This loads the best weights stored by the ES callback
model.load_weights('./working/')

In [ ]:
# Model evaluation, here just as an example done on val set
model.evaluate(val_ds, steps=validation_steps)

In [ ]:
plot_learning_curves(history)

# Training with data augmentation

# Data augmentation placed outside model in the data pipeline, TPU may not support augmentation ops

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
    layers.experimental.preprocessing.RandomContrast(factor=0.2),
])

def data_augment(img, label):
    return data_augmentation(img), label


train_ds = prepare_for_training(train_list_ds, 
                               data_augement_fn=data_augment)

In [ ]:
# Early stopping callback automatically retrieving best weights
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2,
                                                    restore_best_weights=True)

In [ ]:
with strategy.scope():
    model = define_model(3, show_summary=True)

In [ ]:
history = model.fit(train_ds, 
                    steps_per_epoch=steps_per_epoch,
                    epochs=5, 
                    validation_data=val_ds,
                    validation_steps=validation_steps,
                    verbose=1,
                    callbacks=[early_stopping_cb])